In [ ]:
#using the Salmon and Diamond output files from ORP, this code will assign the gene ID found by Diamond to 
#the correct Name/info from Salmon 
import pandas as pd

#load the two files into data frames
salmon_df = pd.read_csv("dero_salmon.csv")
diamond_df = pd.read_csv("dero.ORP.diamond.csv")

#merge the two dataframes based on the Name column
merged_df = pd.merge(salmon_df, diamond_df, on="Name", how="inner")

#selects only certain columns to show up in the final df
output_df = merged_df[["ID","Name", "Length", "EffectiveLength", "TPM", "NumReads"]]

print(output_df)

#saves the resulting df to a csv file
#output_df.to_csv("deroSDmerged.csv", index=False)

In [ ]:
#this code will merge the output summerization files from tximport into a single counts file compatible with DeSeq
import pandas as pd
import numpy as np

file_names = ["chviCounts.csv", "cabrCounts.csv", "stluCounts.csv", 
              "arjaCounts.csv", "lothCounts.csv", 
              "angeCounts.csv", "glsoCounts.csv", 
              "noleCounts.csv", "moblCounts.csv", 
              "mawaCounts.csv", "ptpaCounts.csv", 
              "deroCounts.csv"]

# Create an empty DataFrame to store the merged data
merged_data = pd.DataFrame(columns=["ID"])

# Loop through each file and merge its data with the existing merged data
for file in file_names:
    data = pd.read_csv(file)
    merged_data = pd.merge(merged_data, data, on="ID", how="outer")

# Fill NaN values with 0
merged_data.fillna(0, inplace=True)

print(merged_data)
# Write the merged data to a new CSV file
#merged_data.to_csv("merged_data.csv", index=False)



In [ ]:
#this code will merge the output summerization files from tximport into a single counts file compatible with DeSeq
import pandas as pd
import numpy as np

#create a list of all the file names 
file_names = ["chvi_summerization.csv", "cabr_summerization.csv", "stlu_summerization.csv", "arja_summerization.csv", "loth_summerization.csv", "ange_summerization.csv", "glso_summerization.csv", "nole_summerization.csv", "mobl_summerization.csv", "mawa_summerization.csv", "ptpa_summerization.csv", "dero_summerization.csv"]

#starts by putting the first file into the dataframe
merged_df = pd.read_csv(file_names[0], index_col="ID")

#now the remaining files will be looped through
#the files get added to the main merged df in the end 
for file_name in file_names[1:]:
    #the file is loaded into a df
    df = pd.read_csv(file_name, index_col="ID")
    #file gets merged into the main dataframe
    #missing values are filled with 0 to keep all IDs
    merged_df = merged_df.join(df, how="outer", rsuffix="_" + file_name[:-4]).fillna(0)

#save to a new file... these vaules will be raw/unrounded.. DeSeq needs integers
#merged_df.to_csv("mergedCountFiles.csv", index=True)

merged_df_integers = merged_df.round(0)
print(merged_df_integers)
#merged_df_integers.to_csv("roundedMergedCountFile.csv", index=True)


In [ ]:
#this makes the previous file vaules into boolean.. either the species expresses the gene (1) or it doesn't (0)
import pandas as pd

#reads in the csv file 
df = pd.read_csv('roundedMergedCountFile.csv')

#replaces all vaules in the
#astype converts the boolean T/F to integers 1/0
df.iloc[:, 1:] = (df.iloc[:, 1:] != 0).astype(int)

print(df)
#make a new file with the updated df
df.to_csv('countsFileBoolean.csv', index=False)

In [ ]:
import pandas as pd
df = pd.read_csv('countsFileBoolean.csv')

#iterate over the columns and drop rows with exact value "0"
for col in df.columns:
    df = df[df[col] != 0]

# reset the index after removing rows
df.reset_index(drop=True, inplace=True)

# write the updated DataFrame to a new CSV file
df.to_csv('updated_file_name.csv', index=False)